In [1]:
#mimu4462

In [2]:
import binascii
import threading
from collections import deque

import glob
import serial
import sys
import time
import struct


### connection

In [3]:
import abc


class Connectivity(object):
    __metaclass__ = abc.ABCMeta

    @abc.abstractmethod
    def __init__(self):
        pass

    @abc.abstractmethod
    def open(self, device_params):
        pass

    @abc.abstractmethod
    def send(self, data):
        pass

    @abc.abstractmethod
    def receive(self, len):
        pass

    @abc.abstractmethod
    def close(self):
        pass

    @abc.abstractmethod
    def input_validation(self, *kargs):
        pass

In [4]:
import glob
import serial
import sys


class UsbConnectivity(Connectivity):

    def __init__(self):
        self.usb_serial = None
        super(UsbConnectivity, self).__init__()

    def open(self, device_params):
        """
            Connect to USB port which connect to device
        :param device_params: tuple contains port name(String) and baud rate(integer)
        :raises Exceptoins:
                Failed to open device
        :return: None
        """
        try:
            self.input_validation(device_params[0])
            print("Opening Serial device", device_params[0])
            self.usb_serial = serial.Serial(device_params[0], device_params[1])
        except Exception as ex:
            print("Exception on opening serial: ", ex.message)
            raise ex

        return self.usb_serial

    def send(self, data):
        """
            Send the data to destination device
        :param data:
        :return: None
        """
        self.usb_serial.write(data)
        self.usb_serial.flushOutput()

    def receive(self, length):
        """
            receive the data from connected device
        :param length: size of data to receive
        :return: None
        """
        rcv_buffer = self.usb_serial.read(length)
        return rcv_buffer.encode("hex")

    def close(self):
        """
            Close the socket connection
        :returns:  None
        """
        self.usb_serial.close()

    def input_validation(self, *kargs):
        """
            Check whether port_name is exists or not and its read/write permission
        :param kargs: port name
        :raises EnvironmentError or Exception:
            On unsupported or unknown platforms
            Port name does not exist
        :returns:
            True or False
        """
        if sys.platform.startswith('win'):
            ports = ['COM%s' % (i + 1) for i in range(256)]
        elif sys.platform.startswith('linux') or sys.platform.startswith('cygwin'):
            ports = glob.glob('/dev/tty[A-Za-z]*')
        elif sys.platform.startswith('darwin'):
            ports = glob.glob('/dev/tty.*')
        else:
            raise EnvironmentError('Unsupported platform')

        result = []
        for port in ports:
            try:
                s = serial.Serial(port)
                s.close()
                result.append(port)
            except (OSError, serial.SerialException):
                pass
        if kargs[0] not in result:
            if sys.platform.startswith('linux') or sys.platform.startswith('cygwin') or \
                    sys.platform.startswith('darwin'):
                raise Exception(
                    "Port name does not exists. Please verify the port name and its read/write permission."
                    "\nAvailable ports:- " + str(result))
            else:
                raise Exception("Port name does not exists. Please verify the port name."
                                "\nAvailable ports:- " + str(result))
        return True


### Unities

In [5]:
import string
import random
import serial

In [6]:
plot_descs = {
    1: ["ACC", 'a', '[m/s^2]'],
    2: ["GYRO", "\omega", ' [deg/s]'],
    3: ["MAG", "tesla", ' [mictes]']
}
app = None

def get_connectivity(c_type):
    if str(c_type).upper() == 'USB':
        return UsbConnectivity()
    return None

def get_file_value(filename):  # to read value from a file
    with open(filename, 'r') as filevalue:
        value = filevalue.read()
    return int(value)

def get_y_value(gvalue):  # to get the y values from the file
    y = []
    with open('data/yaxis.txt', 'r') as yvalue:
        for i in range(0, 4):
            y.append(float(yvalue.readline()) * gvalue)
    return y

def get_rate_divider(a):  # to get the rate divider value
    if int(a) == 1:
        return 0
    else:
        return 1 + get_rate_divider(a / 2)

def get_checksum(pkt):  # returns the checksum of packet
    return struct.unpack("!H", pkt[-2:])[0]

def cal_checksum(pkt):  # returns the calculated checksum of packet
    checksum = 0
    a = pkt[:-2].encode('hex')
    x = 0
    y = 2
    for i in range(0, len(a) // 2):
        checksum += int(a[x:y], 16)
        x += 2
        y += 2
    return checksum

def convert_bytes_to_string(byte_array):
    binary_str = ''
    for byte in byte_array:
        binary_str += '{0:08b}'.format(byte)
    return binary_str

def id_generator(size=6, chars=string.ascii_lowercase + string.digits):
    return ''.join(random.choice(chars) for _ in range(size))

def create_logfile(binary_string, select_acm, dlogfile):  # create log file and add header based on selection
    with open(dlogfile, "w") as filedata:
        str_fmt = "%11s\t"
        data_fmt = ["Packet No."]
        for indx in range(0, len(binary_string)):
            if binary_string[indx] == '1':
                acc_gyro = select_acm[indx]
                for j in range(0, len(acc_gyro)):
                    if acc_gyro[j] == '1':
                        axis = j % 3
                        str_fmt += "%10s\t"
                        if j < 3:
                            data_fmt.append("a" + str(axis) + str(indx) + " m/s^2")
                        elif j < 6:
                            data_fmt.append("g" + str(axis) + str(indx) + " deg/s")
                        else:
                            data_fmt.append("m" + str(axis) + str(indx) + " mtesla")
        str_fmt = str_fmt.strip() + "\n"
        hdr_str = str_fmt % tuple(data_fmt)
        filedata.write(hdr_str)
    return filedata

def get_plot_options(select_acm, binary_string):
    plot_opt = set()
    for indx in range(0, len(binary_string)):
        if binary_string[indx] == '1':
            acc_gyro = select_acm[indx]
            for j in range(0, len(acc_gyro)):
                if acc_gyro[j] == '1':
                    if j < 3:
                        plot_opt.add(1)
                    elif j < 6:
                        plot_opt.add(2)
                    else:
                        plot_opt.add(3)
    return list(plot_opt)

def cal_checksum_cmd(cmd):
    checksum = 0
    for i in cmd:
        checksum += i
    return checksum // 256, checksum % 256

def open_device(device, rate):
    try:
        return serial.Serial(device, rate)
    except serial.SerialException as e:
        print("Error opening device:", e)
        open("error", 'w').close()
        sys.exit(1)

def get_ratedevider(a):
    if int(a) == 1:
        return 0
    else:
        return 1 + get_ratedevider(a / 2)

def write_device(device, buffer_c):
    device.write(buffer_c)
    device.flushOutput()

def read_device(device, data_len):
    return device.read(data_len)

def is_hex(s):
    hex_digits = set(string.hexdigits)
    return all(c in hex_digits for c in s)

### MAIN

In [11]:
MAX_FREQ = 500  # Max data rate transmission
g_value = 9.81  # gravity value set
d_graph = 'off line'  # Display graph  #'Real Time' or 'Off Line'
log_data = 1  # Save lag data in file write 1 else 0
plot_graph = 1    # Whether graph is required or not 1 else 0
dlog_file = 'data/bin.txt'  # Data log path directory
out_rate = float(500)       # Data rate (HZ) Note: Maximum supported data rate is 500 Hz when plot_graph is set to 0,
                            # i.e. when output data is logged in file only, without any plot
# conn_params = ('00:A0:50:E4:91:98', 115200)                        # Write hare serial port on your device
# conn_type = 'ble'
conn_params = ('COM3', 460800)  # Write hare serial port on your device
conn_type = 'usb'

NUM_AXIS = 9  # number of axis e.g ax, ay, az, gx, gy, gz , mx, my and mz
FUSE_ITEM = 1  # number of fused item e.g. timestamp
select_acm = '111111111'
runtime = float(10)  # Enter time in second
binary_string = '1'
num_of_selected_imu = 1
app = None

In [12]:
def get_inertial_data(pkt_d):  # returns the inertial data of packet
    return struct.unpack('!I9f', pkt_d)

def insert_data_logfile(file_data, queue_d, num_imu, imu_item, fuse_item, temp_file_t, select_acm_t, binary_string_t):
    # insert data log file and temporary file, this method
    # is run in background
    print("start")
    tmp_file_fmt = "%11d\t" + "%12.3f\t" * (imu_item * num_imu + fuse_item)
    tmp_file_fmt = tmp_file_fmt.strip() + "\n"

    str_fmt = "%11d\t" + "%12.3f\t"

    # str_fmt = str_fmt.strip() + "\n"

    while isRunning:
        while not queue_d.empty():
            data_frm_queue = queue_d.get()
            # print data_frm_queue
            data_str = tmp_file_fmt % tuple(data_frm_queue)
            temp_file_t.write(data_str)  # write data to temporary file
            temp_file_t.flush()
            new_data = []
            ''' Write selectively data to output file - Start  '''
            new_data.append(data_frm_queue[0])  # pkt number
            new_data.append(data_frm_queue[1])  # time (s)
            del data_frm_queue[0]
            del data_frm_queue[0]

            new_str_fmt = str_fmt
            icu_cntry = 0
            for i in range(0, len(binary_string_t)):
                if binary_string_t[i] == '1':
                    ac_gy_ma = select_acm_t
                    acm_len = NUM_AXIS
                    for j in range(0, acm_len):
                        if ac_gy_ma[j] == '1':
                            new_data.append(data_frm_queue[acm_len * icu_cntry + j])
                            new_str_fmt += "%10.3f\t"
                    icu_cntry += 1
            new_str_fmt = new_str_fmt.strip() + "\n"

            data_str = new_str_fmt % tuple(new_data)
            # print hdr_str
            file_data.write(data_str)
            file_data.flush()
            '''   Write selectively data to output file - End   '''
        time.sleep(0.5)

def get_plot_options(select_acm_p, binary_string_p):
    """
    Get Plot options
    :param select_acm_p
    :param binary_string_p:
    :return list of number e.g. [1, 2 , 3 ] mean Plot must
            contains Acc(1), Gyro (2) and Magn(3):
    """
    plot_opt = set()
    for index in range(0, len(binary_string_p)):
        if binary_string_p[index] == '1':
            acc_gyro = select_acm_p
            for j in range(0, len(acc_gyro)):
                if acc_gyro[j] == '1':
                    if j < 3:
                        plot_opt.add(1)
                    elif j < 6:
                        plot_opt.add(2)
                    else:
                        plot_opt.add(3)
    return list(plot_opt)


In [13]:
# Sensor connection setup (simplified)
connectivity_obj = get_connectivity(conn_type)
try:
    com = connectivity_obj.open(conn_params)
except Exception as e:
    print("Connection error:", e)
    sys.exit(1)

Connection error: 'Exception' object has no attribute 'message'


AssertionError: 

In [ ]:
scale_pr_acc = 1.0
scale_pr_gyro = 57.325
scale_pr_mag = 0.6
# generating the command to start normal imu
out_rate_ = MAX_FREQ / float(out_rate)

In [ ]:
cmd = [0x31, 0x16, 0x10, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x00, 0x57]
connectivity_obj.send(cmd)